## Unzipping the TAR File

In [1]:
import wget
import tarfile

In [2]:
# Source: https://www.cs.cmu.edu/~enron/
emails = 'https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz'

In [5]:
wget.download(emails)

100% [......................................................................] 443254787 / 443254787

'enron_mail_20150507.tar.gz'

In [6]:
tf = tarfile.open('enron_mail_20150507.tar.gz')

In [7]:
tf.extractall(r"C:\Users\Aneesha2\Desktop\enron_contents")

## Getting Full Names and Email Adresses

In [2]:
import os
import pandas as pd

In [3]:
# All emails used to construct the full email database
emp_emails = []

In [4]:
# Original content folder
contents = 'C:/Users/Aneesha2/Desktop/enron_contents/maildir'
for emp_folder in os.listdir(contents):
    categories = os.listdir(os.path.join(contents, emp_folder))
    emp_name = False
    for category in categories:
        # Used to loop through all sent emails from each of teh individuals and record their full
        # name using their email adresses
        if 'sent' in category and not emp_name:
            emails_path = f'C:/Users/Aneesha2/Desktop/enron_contents/maildir/{emp_folder}/{category}/1'
            mail = open(emails_path,"r")

            lines = mail.readlines()
            mail.close()
            lines = [line for line in lines if 'From:'  in line]
            if len(lines) != 0:
                email = lines[0].split(' ')[-1][:-1]

                emp_emails.append([emp_folder, email])
                emp_name = True


In [5]:
# Construct dataframe witha name and emails
employees = pd.DataFrame(emp_emails, columns=['folder_name', 'email'])

In [6]:
# Extracting full name from email address
employees['name'] = employees['email'].apply(lambda e: ' '.join(e[:e.index('@')].split('.')))

In [7]:
# Compile of list of accounts that did not have a sent folder
removed_accounts = []

contents = 'C:/Users/Aneesha2/Desktop/enron_contents/maildir'
for emp_folder in employees['folder_name']:
    categories = os.listdir(os.path.join(contents, emp_folder))
    if 'sent' not in categories:
        removed_accounts.append(emp_folder)

In [8]:
# filter for employees with a sent folder
employees['status'] = employees['folder_name'].apply(lambda name: False if name in removed_accounts else True)
employees = employees[employees['status'] == True]

In [10]:
employees = employees.reset_index()
employees = employees[['name', 'email', 'folder_name']]

In [113]:
employees

,name,email,folder_name
0,phillip allen,phillip.allen@enron.com,allen-p
1,john arnold,john.arnold@enron.com,arnold-j
2,suresh raghavan,suresh.raghavan@enron.com,arora-h
3,eric bass,eric.bass@enron.com,bass-e
4,sally beck,sally.beck@enron.com,beck-s
...,...,...,...
84,greg whalley,greg.whalley@enron.com,whalley-g
85,greg whalley,greg.whalley@enron.com,whalley-l
86,zionette vincent,zionette.vincent@enron.com,white-s
87,paul y'barbo,paul.y'barbo@enron.com,ybarbo-p


In [114]:
final_employees = employees[['name', 'email']]

In [115]:
# Looking at the data for 89 users
final_employees

,name,email
0,phillip allen,phillip.allen@enron.com
1,john arnold,john.arnold@enron.com
2,suresh raghavan,suresh.raghavan@enron.com
3,eric bass,eric.bass@enron.com
4,sally beck,sally.beck@enron.com
...,...,...
84,greg whalley,greg.whalley@enron.com
85,greg whalley,greg.whalley@enron.com
86,zionette vincent,zionette.vincent@enron.com
87,paul y'barbo,paul.y'barbo@enron.com


In [116]:
final_employees.to_csv('final_employees.csv')

## Getting all Sent Emails

In [77]:
metadata = pd.DataFrame()
exchange = pd.DataFrame()
bodies = pd.DataFrame()

In [78]:
# starting point
contents = 'C:/Users/Aneesha2/Desktop/enron_contents/maildir'

# pulling employee folders
for emp_folder in employees['folder_name']: #sample
    # sent emails
    emails_path = f'C:/Users/Aneesha2/Desktop/enron_contents/maildir/{emp_folder}/sent/'       
    for email in os.listdir(emails_path):
        # lists out every email
        email_path = emails_path + f'{email}'
        
        # open file
        mail = open(email_path,"r")
        # each line is a line of the text file
        lines = mail.readlines()
        mail.close()

        # 1 element in each of these arrays
        message_id = [line for line in lines if 'Message-ID:' in line]     
        from_email = [line for line in lines if 'From:' in line]    

        # Getting the body of the email
        body_start = [i for i,line in enumerate(lines) if 'X-FileName:' in line][0]
        body = lines[body_start + 2: ]
        body = ' '.join(body)
        
        # list of words
        body = body.split()
        
        # count number of words
        body_len = len(body)

        
        to_email = [line for line in lines if 'To:' in line]   
        emailed = []
        for word in body:
            if '@enron' in str(word):
                emailed.append(word.replace(' ', ''))
        
        if len(to_email) < len(emailed):
            to_email = emailed
            
        exchange = exchange.append({'message_id' : message_id,
                                    'from_email' : from_email,
                                   'to_email' : to_email},
                                  ignore_index = True)

        
    else:
        print(emp_folder)       
print(emp_folder, len(metadata))     

allen-p
arnold-j
arora-h
bass-e
beck-s
benson-r
brawner-s
buy-r
campbell-l
carson-m
cash-m
cuilla-m
dasovich-j
davis-d
dean-c
delainey-d
derrick-j
dickson-s
donohoe-t
dorland-c
ermis-f
farmer-d
fischer-m
fossum-d
gay-r
germany-c
giron-d
grigsby-m
haedicke-m
hayslett-r
heard-m
hernandez-j
hodge-j
horton-s
hyvl-d
jones-t
kaminski-v
kean-s
keavey-p
king-j
kuykendall-t
lavorato-j
lay-k
lenhart-m
lewis-a
lokay-m
love-p
maggi-m
mann-k
martin-t
may-l
mcconnell-m
mckay-b
mclaughlin-e
mims-thurston-p
neal-s
nemec-g
pereira-s
perlingiere-d
presto-k
quenet-j
ring-a
rodrique-r
rogers-b
ruscitti-k
sager-e
sanders-r
scott-s
shackleton-s
shankman-j
shapiro-r
shively-h
skilling-j
smith-m
south-s
stclair-c
sturm-f
symes-k
taylor-m
tholt-j
townsend-j
tycholiz-b
ward-k
weldon-c
whalley-g
whalley-l
white-s
ybarbo-p
zipper-a
zipper-a 57653


In [98]:
# cleaning the exchange table
exchange['from_email'] = exchange['from_email'].apply(lambda data: data[0].replace('From:','').replace('\n', ''))
exchange['message_id'] = exchange['message_id'].apply(lambda data: data[0].replace('Message-ID:','').replace('\n', ''))
exchange['new_to'] = exchange['to_email'].apply(lambda ex : ex[0].replace('To:', '').replace('\n', '').replace(' ', '').split(','))

In [103]:
# reformatting the email excahnges table
new_exchange = pd.DataFrame()
for index, row in exchange.iterrows():
    for item in row['new_to']:
        if '@' in item:
            new_exchange = new_exchange.append({'from_email' : row.from_email,
                                                'message_id' : row.message_id,
                                                'to_email' : item}, 
                                               ignore_index = True)

In [105]:
final_exchange = pd.merge(new_exchange, employees, left_on='to_email', right_on='email', how='inner')
final_exchange = final_exchange[['from_email', 'message_id', 'to_email']]
final_exchange

,from_email,message_id,to_email
0,phillip.allen@enron.com,<5525962.1075855679785.JavaMail.evans@thyme>,christi.nicolay@enron.com
1,phillip.allen@enron.com,<5525962.1075855679785.JavaMail.evans@thyme>,james.steffes@enron.com
2,phillip.allen@enron.com,<5525962.1075855679785.JavaMail.evans@thyme>,jeff.dasovich@enron.com
3,ina.rangel@enron.com,<4650921.1075855679981.JavaMail.evans@thyme>,amanda.huble@enron.com
4,phillip.allen@enron.com,<7226813.1075855681950.JavaMail.evans@thyme>,pallen70@hotmail.com
...,...,...,...
69829,paul.y'barbo@enron.com,<25478099.1075851946978.JavaMail.evans@thyme>,dan.masters@enron.com
69830,paul.y'barbo@enron.com,<25478099.1075851946978.JavaMail.evans@thyme>,rick.sierra@enron.com
69831,paul.y'barbo@enron.com,<25478099.1075851946978.JavaMail.evans@thyme>,greg.curran@enron.com
69832,andy.zipper@enron.com,<9369111.1075851675416.JavaMail.evans@thyme>,chris.edmonds@truequote.com


In [121]:
# Used to look at tevery interaction between upper level management
all_exchanges.to_csv('all_exchanges.csv')